In [0]:
import tensorflow as tf
from tensorflow.keras import layers
from PIL import Image
import os
import pathlib
import numpy as np
import pathlib
import shutil

In [0]:
BASE_PATH = '/dbfs/FileStore/shared_uploads/wu.wenjun@otis.com/0324'
DATA_PATH = BASE_PATH + "/0324data"
# train_dir = pathlib.Path(DATA_PATH+'/training')
# val_dir = pathlib.Path(DATA_PATH+'/val')

# train_dir = dbutils.fs.ls(DATA_PATH+"/train/")
# val_dir = dbutils.fs.ls(DATA_PATH+"/val/")
batch_size = 128
img_height = 120
img_width = 160

resized_height = 120
resized_width = 160

# IMG_SHAPE = (224, 224, 3)
# CLASS_PEOPLE = "/dbfs/FileStore/shared_uploads/wu.wenjun@otis.com/training/people_class"
num_classes = 2
NUM_EPOCHS = 120
seed = 12
log_dir = BASE_PATH + "/logs"

STEPS_PER_EPOCH = 1508 //batch_size

In [0]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
data_augmentation = tf.keras.Sequential([
#   layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomFlip("horizontal"),
  layers.experimental.preprocessing.RandomRotation(0.2),
  layers.experimental.preprocessing.RandomZoom(0.2),
  layers.experimental.preprocessing.RandomContrast(0.2),
#   layers.experimental.preprocessing.RandomContrast(0.2),
])

resize = tf.keras.layers.experimental.preprocessing.Resizing(resized_height, resized_width)

def preprocess_input(image):
    return (image/127.5) - 1

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  DATA_PATH,
  seed=seed,
  image_size=(img_height, img_width)
)
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)

train_ds = train_ds.map(
  lambda x, y: (data_augmentation(x, training=True), y))

train_ds = train_ds.map(
  lambda x, y: (preprocess_input(x), y))


# val_ds = tf.keras.preprocessing.image_dataset_from_directory(
#   DATA_PATH,
#   seed=seed,
#   shuffle=True,
#   validation_split=0.2,
#   subset='validation',
#   image_size=(img_height, img_width)
# )
# val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
# val_ds = val_ds.map(
#   lambda x, y: (preprocess_input(x), y))

Found 10880 files belonging to 2 classes.

In [0]:
def get_model(lr=0.001, alpha=0.35):
  base_model = tf.keras.applications.MobileNetV2(
      input_shape=(img_height, img_width, 3),
      include_top=False, weights='imagenet', alpha=alpha)
  #     base_model.trainable = False
  base_model.trainable = True
  x = base_model.output
  # Add a new classifier layer for transfer learning
  x = tf.keras.layers.GlobalAveragePooling2D()(x)
  x = tf.keras.layers.Dropout(0.2)(x)
  x = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
  #     prediction_layer = tf.keras.layers.Dense(1, activation='softmax')
  model = tf.keras.Model(inputs=base_model.inputs, outputs=x)
  return model

def get_compiled_model(alpha=0.35, lr=0.001):
  model = get_model(lr, alpha)
  #     lr = learning_rate = CustomSchedule(2)
  model.compile(
              optimizer=tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9),
  #                 optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
  return model

In [0]:
def train_and_evaluate(train_ds, val_ds=None, lr=0.001):
  model = get_compiled_model()
  #     model = get_compiled_model_2()
  checkpoint_path = BASE_PATH + "/Model/Saved_model/version-{epoch:04d}.h5py"
  checkpoint_dir = os.path.dirname(checkpoint_path)

  model.save_weights(checkpoint_path.format(epoch=0))

  # 创建一个保存模型权重的回调
  cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                   save_weights_only=False,
                                                   verbose=1)
#   save_best_model = tf.keras.callbacks.ModelCheckpoint(filepath = "/dbfs/FileStore/shared_uploads/wu.wenjun@otis.com/Model2/best_model",
#                                                        monitor='loss', verbose=1, save_best_only=True,
#                                                         save_weights_only=False, mode='min', save_freq='epoch',
#                                                         options=None)

  early_stop = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=10)
  board_callback = tf.keras.callbacks.TensorBoard(
                                log_dir=log_dir,
                                histogram_freq=0,  # How often to log histogram visualizations
                                embeddings_freq=0,  # How often to log embedding visualizations
                                update_freq="epoch",
                                )  # How often to write logs (default: once per epoch)

  #     steps_per_epoch = len(train_ds) // batch_size
  hist = model.fit(train_ds, 
  #                      steps_per_epoch=steps_per_epoch,
             epochs=NUM_EPOCHS,
             validation_data=val_ds,
  #                      class_weight={0:0.3, 1:0.7},
  #                      validation_steps=validation_steps,
             verbose=2,
             callbacks=[
               cp_callback, 
               board_callback, 
               early_stop
  #                        CustomLearningRateScheduler(lr_schedule),
             ])
  return hist,model

In [0]:
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    for gpu in gpus:
#       tf.config.experimental.set_memory_growth(gpu, True)
      print('yes')
#     tf.config.experimental.set_virtual_device_configuration(
#         gpus[0],
#         [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
yes
yes
2 Physical GPUs, 2 Logical GPUs

In [0]:
tf.debugging.set_log_device_placement(True)
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
  hist,model = train_and_evaluate(train_ds)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
WARNING:tensorflow:`input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
Downloading data from https://storage.googleapis.com/tensorflow/keras-applications/mobilenet_v2/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_0.35_224_no_top.h5
 8192/2019640 [..............................] - ETA: 0s2023424/2019640 [==============================] - 0s 0us/step
Epoch 1/120
WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/tensorflow/python/data/ops/multi_device_iterator_ops.py:601: get_next_as_optional (from tensorflow.python.data.ops.iterator_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.
INFO:tensorflow:batch_all_reduce: 158 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 158 all-reduces with algorithm = nccl, num_packs = 1
WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0661s vs `on_train_batch_end` time: 0.4241s). Check your callbacks.

Epoch 00001: saving model to /dbfs/FileStore/shared_uploads/wu.wenjun@otis.com/0324/Model/Saved_model/version-0001.h5py
WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/tensorflow/python/training/tracking/tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/tensorflow/python/training/tracking/tracking.py:111: Layer.updates (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.

In [0]:
model.save(BASE_PATH + '/Model/Best_Model')

In [0]:
!tar -cvf /dbfs/FileStore/shared_uploads/wu.wenjun@otis.com/0324/Model/Best_Model.tar /dbfs/FileStore/shared_uploads/wu.wenjun@otis.com/0324/Model/Best_Model